In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from pathlib import Path
from dotenv import load_dotenv,find_dotenv

from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.storage import LocalFileStore,InMemoryStore
from langchain_community.vectorstores import FAISS

c:\Users\MSI\anaconda3\envs\lcnenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
embeddingModelsOAI=OpenAIEmbeddings()
embeddingModelsSTS=SentenceTransformerEmbeddings()

C:\Users\MSI\AppData\Local\Temp\ipykernel_7840\4026543667.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddingModelsSTS=SentenceTransformerEmbeddings()
C:\Users\MSI\AppData\Local\Temp\ipykernel_7840\4026543667.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddingModelsSTS=SentenceTransformerEmbeddings()


<h3>Embed Documents takes in a list of Strings</h3>

In [4]:
embeddings=embeddingModelsOAI.embed_documents(texts=[
                                                "My Son's name is Eklavya",
                                                "He is 18 months old"]
                                             )

In [5]:
embeddings

[[0.004032173193991184,
  -0.009886577725410461,
  -0.010229053907096386,
  -0.0084585165604949,
  -0.020626114681363106,
  0.040735285729169846,
  -0.027656571939587593,
  -0.008503749035298824,
  -0.015585899353027344,
  0.0024167189840227365,
  0.012665159069001675,
  -0.016089921817183495,
  0.019928239285945892,
  -0.004074174910783768,
  -0.00902069453150034,
  -0.006739673670381308,
  0.020781198516488075,
  -0.02682945877313614,
  0.029233254492282867,
  -0.010687842965126038,
  -0.008032036945223808,
  0.010920468717813492,
  0.003174366895109415,
  -0.004885132424533367,
  -0.021130137145519257,
  0.01829986274242401,
  -0.003395684063434601,
  -0.01660686545073986,
  0.006584590300917625,
  -0.0013844439527019858,
  -0.00041355620487593114,
  -0.008846225216984749,
  -0.0067719826474785805,
  -0.010817079804837704,
  -0.032826025038957596,
  -0.0008642678731121123,
  -0.004991752561181784,
  -0.005641165189445019,
  0.02127229794859886,
  -0.003515227697789669,
  0.020742427

In [6]:
len(embeddings)

2

In [7]:
len(embeddings[0])

1536

<h3>Embed Query takes in a single string</h3>

In [8]:
embeddings=embeddingModelsSTS.embed_query(text="My Son's name is Eklavya. He is 18 Months old")

In [9]:
embeddings

[0.05749928951263428,
 -0.013831495307385921,
 -0.0435388907790184,
 0.0068318601697683334,
 0.03582257777452469,
 -0.0289433766156435,
 -0.032146573066711426,
 0.017171509563922882,
 0.016495291143655777,
 0.06891708076000214,
 0.028984112665057182,
 -0.06927915662527084,
 0.02675733156502247,
 -0.018743036314845085,
 -0.05739450454711914,
 -0.02812318503856659,
 -0.0009407802135683596,
 -0.019779589027166367,
 0.05335355922579765,
 -0.01864325813949108,
 -0.028344472870230675,
 0.010000643320381641,
 0.006351907271891832,
 0.043156806379556656,
 0.02063271962106228,
 0.023404613137245178,
 -0.03515417501330376,
 0.013309652917087078,
 -0.01540792640298605,
 -0.04030275344848633,
 -0.035501956939697266,
 -0.020003724843263626,
 -0.009244359098374844,
 0.06103907525539398,
 1.461730221308244e-06,
 0.03680436313152313,
 -0.005678417626768351,
 -0.030712541192770004,
 -0.008487763814628124,
 -0.07723864167928696,
 0.017104914411902428,
 -0.05201288312673569,
 -0.01568351499736309,
 -0.02

In [9]:
len(embeddings)

768

In [10]:
embeddingModelsOAI.model

'text-embedding-ada-002'

<h3>Caching of Embeddings</h3>

In [16]:
filePath=Path("data").joinpath("letters").joinpath("state_of_the_union.txt")

In [11]:
tempStore=LocalFileStore(root_path="./cache/")

In [12]:
# The main supported way to initialize a CacheBackedEmbeddings is from_bytes_store
cachedEmbedder=CacheBackedEmbeddings.from_bytes_store(
                                                underlying_embeddings=embeddingModelsOAI,
                                                document_embedding_cache=tempStore, 
                                                namespace="sentenceEmbeddings"
                                                )

In [13]:
# So far empty
list(tempStore.yield_keys())

['sentenceEmbeddings01dbc21f-5e4c-5fb5-8d13-517dbe7a32d4',
 'sentenceEmbeddings059eb9ff-c4c8-5ceb-8bf9-0d3d02a92b44',
 'sentenceEmbeddings0fc1ede2-407a-5e14-8f8f-5642214263f5',
 'sentenceEmbeddings1297d37a-2bc1-5e19-bf13-6c950f075062',
 'sentenceEmbeddings17a6727d-8916-54eb-b196-ec9c9d6ca472',
 'sentenceEmbeddings1d203ac6-0e5c-5e99-9c20-884b3206ba18',
 'sentenceEmbeddings20c8f906-bea3-5e8c-b01a-e5ecfa990007',
 'sentenceEmbeddings24610cdc-5e53-577d-8142-5f9c16afbd22',
 'sentenceEmbeddings280a9fbe-0d67-50df-9796-d781d74a0ea4',
 'sentenceEmbeddings2b51a7dd-babe-5c36-8930-3baa92a1130a',
 'sentenceEmbeddings305efb5c-3f01-5657-bcf2-2b92fb1747ca',
 'sentenceEmbeddings3f32125d-8e2c-5e8f-8ea1-0e4fe633c9f5',
 'sentenceEmbeddings3f7b9f1f-79ae-55e3-966a-d0ec952476ed',
 'sentenceEmbeddings464862c8-03d2-5854-b32c-65a075e612a2',
 'sentenceEmbeddings4e5734dd-6470-5d9e-a7b8-468ebc563c4e',
 'sentenceEmbeddings5990258b-0781-5651-8444-c69cb5b6da3d',
 'sentenceEmbeddings5ba09d7e-6a58-5c76-b038-5d8636e5ea25

In [17]:
rawDocument=TextLoader(file_path=filePath,encoding="utf-8").load()
splittedDocuments=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=10).split_documents(documents=rawDocument)

<H2>Creating the Vectorstore</H2>

In [20]:
db=FAISS.from_documents(documents=splittedDocuments,embedding=cachedEmbedder)

In [18]:
# Will not be empty
list(tempStore.yield_keys())

['sentenceEmbeddings01dbc21f-5e4c-5fb5-8d13-517dbe7a32d4',
 'sentenceEmbeddings059eb9ff-c4c8-5ceb-8bf9-0d3d02a92b44',
 'sentenceEmbeddings0fc1ede2-407a-5e14-8f8f-5642214263f5',
 'sentenceEmbeddings1297d37a-2bc1-5e19-bf13-6c950f075062',
 'sentenceEmbeddings17a6727d-8916-54eb-b196-ec9c9d6ca472',
 'sentenceEmbeddings1d203ac6-0e5c-5e99-9c20-884b3206ba18',
 'sentenceEmbeddings20c8f906-bea3-5e8c-b01a-e5ecfa990007',
 'sentenceEmbeddings24610cdc-5e53-577d-8142-5f9c16afbd22',
 'sentenceEmbeddings280a9fbe-0d67-50df-9796-d781d74a0ea4',
 'sentenceEmbeddings2b51a7dd-babe-5c36-8930-3baa92a1130a',
 'sentenceEmbeddings305efb5c-3f01-5657-bcf2-2b92fb1747ca',
 'sentenceEmbeddings3f32125d-8e2c-5e8f-8ea1-0e4fe633c9f5',
 'sentenceEmbeddings3f7b9f1f-79ae-55e3-966a-d0ec952476ed',
 'sentenceEmbeddings464862c8-03d2-5854-b32c-65a075e612a2',
 'sentenceEmbeddings4e5734dd-6470-5d9e-a7b8-468ebc563c4e',
 'sentenceEmbeddings5990258b-0781-5651-8444-c69cb5b6da3d',
 'sentenceEmbeddings5ba09d7e-6a58-5c76-b038-5d8636e5ea25

<h3>Swapping the Bytestore</h3>

In [19]:
memStore=InMemoryStore()

In [20]:
cached_embedder=CacheBackedEmbeddings.from_bytes_store(underlying_embeddings=embeddingModelsSTS,
                                                       document_embedding_cache=memStore,
                                                       namespace="sentenceEmbeddingsMemory")

In [21]:
db=FAISS.from_documents(documents=splittedDocuments,embedding=cachedEmbedder)